In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import os

In [7]:
%cd /content/drive/MyDrive/Yeast

/content/drive/MyDrive/Yeast


In [8]:
test_folder_path = 'BackTest/2h'
mask_folder_path = 'BackTest/mask2h'
watershed_mask_folder = 'BackTest/2h_watershed'

In [9]:
origin_masks = os.listdir(mask_folder_path)

for mask in origin_masks:
  img = cv.imread(os.path.join(mask_folder_path, mask))
  file_name = mask.split('.')[0]
  name = file_name+ 'watershed.png'

  # imgplto = plt.imshow(img)
  gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

  # imgplt = plt.imshow(gray)
  ret, thresh = cv.threshold(gray, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)
  thresh = cv.bitwise_not(thresh)

  # imgplt1 = plt.imshow(thresh)
  kernel = np.ones((3, 3), np.uint8)
  opening = cv.morphologyEx(thresh, cv.MORPH_OPEN, kernel, iterations=2)
  sure_bg = cv.dilate(opening, kernel, iterations=3)
  dist_transform = cv.distanceTransform(opening, cv.DIST_L2, 5)
  ret, sure_fg = cv.threshold(dist_transform, 0.5 * dist_transform.max(), 255, 0)
  sure_fg = np.uint8(sure_fg)
  unknown = cv.subtract(sure_bg, sure_fg)
  # imgplt2 = plt.imshow(unknown)
  #cv.imwrite(os.path.join(unknown_mask_path, name), unknown)

  ret, markers = cv.connectedComponents(sure_fg)
  markers = markers + 1
  markers[unknown == 255] = 0
  markers = cv.watershed(img, markers)
  # imgplt3 = plt.imshow(markers)

  img[markers == -1] = [0, 0, 255]
  # imgplt4 = plt.imshow(img)

  cv.imwrite(os.path.join(watershed_mask_folder, name), img)

In [ ]:
cv.imwrite('/content/drive/MyDrive/2022.2/Out_mask_Watershed/test2.png',img)

False